In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
from time import sleep

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison

## Definição dos tratamentos

In [2]:
df = pd.DataFrame(gpd.read_file('../datasets/area_estudo_teste1/sections/baianinha/canopy_ARD_sampled_baianinha.geojson').drop(['geometry'], axis=1))

# df_raw = df_raw.loc[df_raw['rep']==1]
# df_raw.rename(columns={'tempmean': 'temp'},inplace=True)
# df_raw = df_raw.groupby('name', as_index=False).mean()
# df_raw['trat'] = [x[:2] for x in df_raw['name'].values]

# df = df_raw.copy()
df

,fid,label_id,row,id_row,greening,termal_count,termal_sum,termal_mean,termal_median,termal_stdev,...,pca4_mean,pca4_median,pca4_stdev,pca4_min,pca4_max,pca4_range,pca4_minority,pca4_majority,pca4_variety,pca4_variance
0,1531,024-0006,24,6,1.0,1294.0,42803.979397,33.078809,32.671074,1.649757,...,0.002165,0.001712,0.003205,-0.005221,0.015644,0.020865,-0.005221,-0.005221,766.0,0.000010
1,1538,030-0044,30,44,1.0,1290.0,50760.862091,39.349505,39.360111,1.865633,...,0.001413,0.001260,0.003610,-0.007543,0.014796,0.022338,-0.007543,-0.007543,762.0,0.000013
2,1620,028-0047,28,47,1.0,1502.0,53987.347061,35.943640,35.234539,2.579985,...,0.004815,0.004754,0.003075,-0.003783,0.020777,0.024560,-0.003783,-0.003783,890.0,0.000009
3,1621,026-0043,26,43,1.0,1325.0,47255.736591,35.664707,35.426250,2.040055,...,0.002049,0.002307,0.003247,-0.009210,0.012887,0.022096,-0.009210,-0.009210,784.0,0.000011
4,1630,027-0064,27,64,1.0,988.0,34327.370571,34.744302,34.315372,1.837538,...,0.002026,0.001836,0.003480,-0.006207,0.016693,0.022900,-0.006207,-0.006207,585.0,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,7443,021-0070,21,70,1.0,973.0,34173.285006,35.121567,34.472805,2.257829,...,0.003415,0.003281,0.003001,-0.005791,0.010724,0.016516,-0.005791,-0.005791,577.0,0.000009
2072,7445,021-0068,21,68,1.0,926.0,32758.673321,35.376537,35.153622,1.997268,...,0.008162,0.008370,0.004464,-0.008354,0.022779,0.031133,-0.008354,-0.008354,550.0,0.000020
2073,7447,018-0069,18,69,1.0,609.0,22998.449787,37.764285,37.570312,1.337039,...,-0.000862,-0.000715,0.001777,-0.005886,0.003697,0.009583,-0.005886,-0.005886,353.0,0.000003
2074,7449,019-0071,19,71,1.0,746.0,28413.030224,38.087172,37.839745,1.786820,...,0.002588,0.002505,0.001747,-0.002238,0.007983,0.010221,-0.002238,-0.002238,440.0,0.000003


In [3]:
df_teste = df.loc[:,[x for x in df.columns if '_mean' in x or x in ['label_id', 'greening']]]

df_teste

,label_id,greening,termal_mean,ndvi_mean,ndvie_mean,tvi_mean,sipi_mean,vari_mean,lci_mean,green_mean,red_mean,rededge_mean,nir_mean,pca1_mean,pca2_mean,pca3_mean,pca4_mean
0,024-0006,1.0,33.078809,0.827298,0.204242,15.413842,0.769888,0.755492,0.309965,0.041353,0.024142,0.177577,0.269565,0.343862,0.083481,-0.024931,0.002165
1,030-0044,1.0,39.349505,0.847448,0.197520,16.385581,0.781758,0.781160,0.303243,0.042161,0.021771,0.190967,0.281270,0.381733,0.097294,-0.017554,0.001413
2,028-0047,1.0,35.943640,0.847999,0.201193,16.580863,0.770824,0.779380,0.307451,0.045566,0.021617,0.190571,0.281999,0.375102,0.094457,-0.018503,0.004815
3,026-0043,1.0,35.664707,0.830370,0.170864,15.745522,0.758462,0.769569,0.266199,0.045399,0.023552,0.193823,0.271414,0.357581,0.100392,-0.011964,0.002049
4,027-0064,1.0,34.744302,0.820260,0.192007,14.962563,0.759071,0.747956,0.292143,0.042733,0.024920,0.179315,0.262420,0.333034,0.089456,-0.018439,0.002026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,021-0070,1.0,35.121567,0.824429,0.214992,16.580631,0.765482,0.744814,0.321850,0.046543,0.027991,0.189768,0.291966,0.341116,0.088810,-0.027317,0.003415
2072,021-0068,1.0,35.376537,0.831741,0.242693,16.355707,0.759572,0.738354,0.355881,0.048079,0.025604,0.175195,0.283216,0.336485,0.086114,-0.032307,0.008162
2073,018-0069,1.0,37.764285,0.741552,0.155063,13.319998,0.706753,0.665030,0.233811,0.047865,0.037793,0.185696,0.253078,0.310864,0.100510,-0.011847,-0.000862
2074,019-0071,1.0,38.087172,0.794443,0.146039,15.309321,0.728131,0.734388,0.228423,0.051405,0.031195,0.203713,0.272876,0.338914,0.102101,-0.007790,0.002588


In [4]:
df_long = pd.melt(df_teste, id_vars=['greening', 'label_id'])
df_long

,greening,label_id,variable,value
0,1.0,024-0006,termal_mean,33.078809
1,1.0,030-0044,termal_mean,39.349505
2,1.0,028-0047,termal_mean,35.943640
3,1.0,026-0043,termal_mean,35.664707
4,1.0,027-0064,termal_mean,34.744302
...,...,...,...,...
31135,1.0,021-0070,pca4_mean,0.003415
31136,1.0,021-0068,pca4_mean,0.008162
31137,1.0,018-0069,pca4_mean,-0.000862
31138,1.0,019-0071,pca4_mean,0.002588


In [ ]:
sns.boxplot("variable", hue="greening", y="value", data=df_long)

C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\seaborn\_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:xlabel='variable', ylabel='value'>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
df = sns.load_dataset('tips')

sns.boxplot(x="day", y="total_bill", hue="smoker", data=df, palette="Set1", width=0.5)

In [ ]:
df_filtered = df_teste.iloc[:,2:]
X = (df_filtered-df_filtered.min())/(df_filtered.max()-df_filtered.min())
X

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=2)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

In [ ]:
col_to_group='greening'
col_for_data='termal_mean'

samples =  [cols[1] for cols in df.groupby(col_to_group)[col_for_data]]


# samples = [cols[1] for cols in df.groupby(col_to_group)[col_for_data]]
samples

## Teste das pressuposições

### - Igualdade das variâncias

In [ ]:
print('.: Teste de variância :.')
stats.bartlett(*samples)

A 5%, todas as variâncias populacionais são iguais, visto que valor-p = 0.751

### - Normalidade das populações

In [ ]:
print('.: Teste de normalidade :.')
for i in samples:
    print(stats.shapiro(i))

Portanto, considerando-se 5% de significância, não há razões para discordar que todas as amostras são provenientes de populações com distribuição normal, uma vez que todos os valores-p são maiores que o nível de significância adotado. Ou seja, H[zero] é aceita.

## ANOVA

In [ ]:
stats.f_oneway(*samples)

### Tabela

## Teste Turkey

In [ ]:
# Read data.  Comment out the next ones to use random data.  
# df=pd.read_excel('anova_test.xlsx')
# n=1000
# df = pd.DataFrame(columns=['Groups','Data'],index=np.arange(n))
# df['Groups']=np.random.randint(1, 4,size=n)
# df['Data']=df['Groups']*np.random.random_sample(size=n)

# define columns for data and then grouping
# col_to_group='trat'
# col_for_data='temp'


# df_03_08_21
# df_20_08_20
# df_29_09_20
# df_30_09_20_m
# df_30_09_20_t

# df = df_30_09_20_m.copy()



#Now take teh data and regroup for anova
samples = [cols[1] for cols in df.groupby(col_to_group)[col_for_data]]    #I am not sure how this works but it makes an numpy array for each group 
f_val, p_val = stats.f_oneway(*samples)  # I am not sure what this star does but this passes all the numpy arrays correctly
print('F value: {:.3f}, p value: {:.50f}\n'.format(f_val, p_val))

# this if statement can be uncommmented if you don't won't to go furhter with out p<0.05
#if p_val<0.05:    #If the p value is less than 0.05 it then does the tukey
mod = MultiComparison(df[col_for_data], df[col_to_group])
thsd=mod.tukeyhsd()
#print(mod.tukeyhsd())

#this is a function to do Piepho method.  AN Alogrithm for a letter based representation of al-pairwise comparisons.  
tot=len(thsd.groupsunique)
#make an empty dataframe that is a square matrix of size of the groups. #set first column to 1
df_ltr=pd.DataFrame(np.nan, index=np.arange(tot),columns=np.arange(tot))
df_ltr.iloc[:,0]=1
count=0
df_nms = pd.DataFrame('', index=np.arange(tot), columns=['names'])  # I make a dummy dataframe to put axis labels into.  sd stands for signifcant difference

with tqdm(total=tot) as pbar:
    pbar.set_description("Processando")
    sleep(0.1)
    
    for i in np.arange(tot):   #I loop through and make all pairwise comparisons. 
        for j in np.arange(i+1,tot):
            #print('i=',i,'j=',j,thsd.reject[count])
            if thsd.reject[count]==True:
                for cn in np.arange(tot):
                    if df_ltr.iloc[i,cn]==1 and df_ltr.iloc[j,cn]==1: #If the column contains both i and j shift and duplicat
                        df_ltr=pd.concat([df_ltr.iloc[:,:cn+1],df_ltr.iloc[:,cn+1:].T.shift().T],axis=1)
                        df_ltr.iloc[:,cn+1]=df_ltr.iloc[:,cn]
                        df_ltr.iloc[i,cn]=0
                        df_ltr.iloc[j,cn+1]=0
                    #Now we need to check all columns for abosortpion.
                    for cleft in np.arange(len(df_ltr.columns)-1):
                        for cright in np.arange(cleft+1,len(df_ltr.columns)):
                            if (df_ltr.iloc[:,cleft].isna()).all()==False and (df_ltr.iloc[:,cright].isna()).all()==False: 
                                if (df_ltr.iloc[:,cleft]>=df_ltr.iloc[:,cright]).all()==True:  
                                    df_ltr.iloc[:,cright]=0
                                    df_ltr=pd.concat([df_ltr.iloc[:,:cright],df_ltr.iloc[:,cright:].T.shift(-1).T],axis=1)
                                if (df_ltr.iloc[:,cleft]<=df_ltr.iloc[:,cright]).all()==True:
                                    df_ltr.iloc[:,cleft]=0
                                    df_ltr=pd.concat([df_ltr.iloc[:,:cleft],df_ltr.iloc[:,cleft:].T.shift(-1).T],axis=1)

            count+=1
        pbar.update(1)

    #I sort so that the first column becomes A        
    df_ltr=df_ltr.sort_values(by=list(df_ltr.columns),axis=1,ascending=False)

    # I assign letters to each column
    for cn in np.arange(len(df_ltr.columns)):
        df_ltr.iloc[:,cn]=df_ltr.iloc[:,cn].replace(1,chr(97+cn)) 
        df_ltr.iloc[:,cn]=df_ltr.iloc[:,cn].replace(0,'')
        df_ltr.iloc[:,cn]=df_ltr.iloc[:,cn].replace(np.nan,'') 

    #I put all the letters into one string
    df_ltr=df_ltr.astype(str)
    # df_ltr.sum(axis=1)
    #print(df_ltr)
    #print('\n')
    #print(df_ltr.sum(axis=1))
    
#Now to plot like R with a violing plot
fig,ax=plt.subplots(figsize=(8,6))
df.boxplot(column=col_for_data, by=col_to_group,ax=ax,fontsize=16,showmeans=False
                    ,boxprops=dict(linewidth=1.0),whiskerprops=dict(linewidth=1.0))  #This makes the boxplot

ax.set_ylim([20,50])

grps=pd.unique(df[col_to_group].values)   #Finds the group names
grps.sort() # This is critical!  Puts the groups in alphabeical order to make it match the plotting

props=dict(facecolor='white',alpha=1)
for i,grp in enumerate(grps):   #I loop through the groups to make the scatters and figure out the axis labels. 

    x = np.random.normal(i+1, 0.15, size=len(df[df[col_to_group]==grp][col_for_data]))
#     ax.scatter(x,df[df[col_to_group]==grp][col_for_data],alpha=0.7,s=4)
    name="{}\navg={:0.2f}\n(n={})".format(grp
                            ,df[df[col_to_group]==grp][col_for_data].mean()
                            ,df[df[col_to_group]==grp][col_for_data].count())
    
    name="{}".format(grp)
    
    df_nms['names'][i]=name 
#     ax.text(i+1,ax.get_ylim()[1]*1.01,df_ltr.sum(axis=1)[i],fontsize=10,verticalalignment='top',horizontalalignment='center',bbox=props)


ax.set_xticklabels(df_nms['names'],rotation=0,fontsize=12)
ax.set_xlabel('Tratamentos', fontsize=12)
ax.set_ylabel('Temperatura (C)', fontsize=12)
ax.set_title('')
fig.suptitle('');

In [ ]:
with tqdm(total=tot) as pbar:
    pbar.set_description("Processando")
    sleep(0.1)
    
    for i in np.arange(tot):   #I loop through and make all pairwise comparisons. 
        for j in np.arange(i+1,tot):
            #print('i=',i,'j=',j,thsd.reject[count])
            if thsd.reject[count]==True:
                for cn in np.arange(tot):
                    if df_ltr.iloc[i,cn]==1 and df_ltr.iloc[j,cn]==1: #If the column contains both i and j shift and duplicat
                        df_ltr=pd.concat([df_ltr.iloc[:,:cn+1],df_ltr.iloc[:,cn+1:].T.shift().T],axis=1)
                        df_ltr.iloc[:,cn+1]=df_ltr.iloc[:,cn]
                        df_ltr.iloc[i,cn]=0
                        df_ltr.iloc[j,cn+1]=0
                    #Now we need to check all columns for abosortpion.
                    for cleft in np.arange(len(df_ltr.columns)-1):
                        for cright in np.arange(cleft+1,len(df_ltr.columns)):
                            if (df_ltr.iloc[:,cleft].isna()).all()==False and (df_ltr.iloc[:,cright].isna()).all()==False: 
                                if (df_ltr.iloc[:,cleft]>=df_ltr.iloc[:,cright]).all()==True:  
                                    df_ltr.iloc[:,cright]=0
                                    df_ltr=pd.concat([df_ltr.iloc[:,:cright],df_ltr.iloc[:,cright:].T.shift(-1).T],axis=1)
                                if (df_ltr.iloc[:,cleft]<=df_ltr.iloc[:,cright]).all()==True:
                                    df_ltr.iloc[:,cleft]=0
                                    df_ltr=pd.concat([df_ltr.iloc[:,:cleft],df_ltr.iloc[:,cleft:].T.shift(-1).T],axis=1)

            count+=1
        pbar.update(1)

    #I sort so that the first column becomes A        
    df_ltr=df_ltr.sort_values(by=list(df_ltr.columns),axis=1,ascending=False)

    # I assign letters to each column
    for cn in np.arange(len(df_ltr.columns)):
        df_ltr.iloc[:,cn]=df_ltr.iloc[:,cn].replace(1,chr(97+cn)) 
        df_ltr.iloc[:,cn]=df_ltr.iloc[:,cn].replace(0,'')
        df_ltr.iloc[:,cn]=df_ltr.iloc[:,cn].replace(np.nan,'') 

    #I put all the letters into one string
    df_ltr=df_ltr.astype(str)
    # df_ltr.sum(axis=1)
    #print(df_ltr)
    #print('\n')
    #print(df_ltr.sum(axis=1))

In [ ]:
#Now to plot like R with a violing plot
fig,ax=plt.subplots(figsize=(15,8))
df.boxplot(column=col_for_data, by=col_to_group,ax=ax,fontsize=16,showmeans=False
                    ,boxprops=dict(linewidth=2.0),whiskerprops=dict(linewidth=2.0))  #This makes the boxplot

# ax.set_ylim([-10,20])

grps=pd.unique(df[col_to_group].values)   #Finds the group names
grps.sort() # This is critical!  Puts the groups in alphabeical order to make it match the plotting

props=dict(facecolor='white',alpha=1)
for i,grp in enumerate(grps):   #I loop through the groups to make the scatters and figure out the axis labels. 

    x = np.random.normal(i+1, 0.15, size=len(df[df[col_to_group]==grp][col_for_data]))
    ax.scatter(x,df[df[col_to_group]==grp][col_for_data],alpha=0.5,s=4)
    name="{}\navg={:0.2f}\n(n={})".format(grp
                            ,df[df[col_to_group]==grp][col_for_data].mean()
                            ,df[df[col_to_group]==grp][col_for_data].count())
    df_nms['names'][i]=name 
    ax.text(i+1,ax.get_ylim()[1]*1.01,df_ltr.sum(axis=1)[i],fontsize=10,verticalalignment='top',horizontalalignment='center',bbox=props)


ax.set_xticklabels(df_nms['names'],rotation=0,fontsize=12)
ax.set_xlabel('Tratamentos', fontsize=12)
ax.set_ylabel('Temperatura (C)', fontsize=12)
ax.set_title('')
fig.suptitle('');

### PCA

In [ ]:
from osgeo import gdal
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from rasterio.plot import show

from scipy import stats
from scipy.cluster.vq import *
import statsmodels.api as sm
import pandas as pd
from scipy import ndimage, misc
from sklearn.decomposition import PCA
from sklearn.preprocessing import *
from scipy.ndimage import filters
from scipy.ndimage import gaussian_filter

In [ ]:
image_data = gdal.Open('../imgs/20_08_20/cliped/pca.tif')
image_data_arr = image_data.ReadAsArray()
image_data_arr[image_data_arr == -999] = np.nan

In [ ]:
# Reshaping dimension

# get the format of the orignial data
n_samples, nx, ny = image_data_arr.shape

# reshape the three dimension to two
image_data_arr_2_dim= image_data_arr.reshape(n_samples, nx*ny)
image_data_arr.shape, image_data_arr_2_dim.shape

In [ ]:
# Determine the number of components
pca = PCA(n_components = 2)

# Scale the 2 dimensional data
image_data_arr_2_dim_scaled = MinMaxScaler().fit_transform(image_data_arr_2_dim)

# run the pca
pca.fit(image_data_arr_2_dim_scaled)

In [ ]:
# Fit the pca to the scaled data
pca_scaled_img = pca.fit_transform(image_data_arr_2_dim_scaled)

# Inverse Components to Orignial  2 space
image_inversed = pca.inverse_transform(pca_scaled_img)

image_scaled = MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(image_inversed)

# PCA Reshape back to the 3 D comp
Components = image_scaled.reshape((n_samples,nx,ny))

# Check the dimensions
Components.shape, image_inversed.shape

In [ ]:
PCA_1 =Components[0]
PCA_2 =Components[1]
PCA_1.shape

In [ ]:
fig, ((ax1, ax2,ax3), (ax4, ax5, ax6))= plt.subplots(2,3, figsize=(20,16))

show((PCA_1), ax=ax1, cmap='gray', title='Component 1')
show((PCA_2), ax=ax2, cmap='gray', title='Component 2')

In [ ]:
pc_img = np.stack((PCA_1,PCA_2),1).reshape(2, PCA_1.shape[0],PCA_1.shape[1])

In [ ]:
pc_img.shape

In [ ]:
# Set Dimensions
rows = image_data.RasterXSize
cols = image_data.RasterYSize
bands = 2

# Set the output path
output_fname = '../imgs/20_08_20/cliped/pca_ready.tif'

# Get the projection 
# and transfromation of original image
geo_transform=image_data.GetGeoTransform()
proj=image_data.GetProjection()

In [ ]:
def arrayRaster(fname, data, geo_transform, projection, data_type=gdal.GDT_Byte):
    # Get GeoTIFF driver
    driver = gdal.GetDriverByName('GTiff')
    #pass parameters to the data
    dataset = driver.Create(fname, rows, cols, bands, gdal.GDT_Float32)
    # set transformation and projection
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)

    # write each array to each band
    band = dataset.GetRasterBand(1).WriteArray(PCA_1)
    band = dataset.GetRasterBand(2).WriteArray(PCA_2)
#     band = dataset.GetRasterBand(3).WriteArray(PCA_3)

# Create the Raster
arrayRaster(output_fname, pc_img, geo_transform, proj)